## AMLD Workshop Notebook 1

In this notebook, you will learn how to connect to a Tune Insight instance from Chorus, create a project, load data, and run a basic analysis.

## 🎯 OBJECTIVE

Validate secure data access and establish a federated project context by connecting CHORUS to a Tune Insight instance and running a first controlled analysis.

<div style="background-color: rgba(182, 255, 18, 0.15); border-left: 5px solid #B6FF12; padding: 15px; margin: 10px 0;">
<h3> Real-world context</h3>
<p>Before any advanced modeling takes place, clinical research projects typically begin with controlled validation steps: confirming that data is accessible, permissions are correctly enforced, and cross-institution collaboration mechanisms function as expected. In federated settings, this includes verifying that each participating institution can contribute to a shared computation without exposing raw data.

CHORUS provides the secure execution environment in which this work happens, while Tune Insight enables federated coordination across institutions. Together, they allow researchers to move from isolated local experimentation to collaborative analysis without breaching data sovereignty or compliance constraints.
</p>
</div>

## ⚙️ IMPLEMENTATION

### Setup

In [ ]:
import pandas as pd
import uuid

from tuneinsight import Diapason  # The Tune Insight client

#### Connecting to the instance

In [ ]:
from amld_setup import *  # This defines variables to connect to the instance

# TODO: Please update the credentials below with those provided to you.
%env TI_USERNAME=amld-workshopX@tuneinsight.com
%env TI_PASSWORD=AMLD_workshop_X

In [ ]:
client = Diapason.from_env()

In [ ]:
client.healthcheck()

#### Create and share the project

Projects are the main unit of collaboration in Tune Insight projects. In a project, you will define the computation to run in a federated setting, and set the datasource used by your instance. Other participants will also choose the data used by their instance. Once everything is set up, the federated analysis can be run using data from all instances, without centralizing the data.

In [ ]:
PROJECT_NAME = f"project-1-{uuid.uuid4()}"

project = client.new_project(name=PROJECT_NAME, clear_if_exists=True)
project.share()

## Load the dataset

The data that we will use in this workshop was provided to you as `data_0.csv`. We'll go into the data in more detail in notebook 2. In this notebook, we will simply load the data and use it for a very simple analysis to make sure everything works.

In [ ]:
data_path = "data/data_0.csv"

In [ ]:
# You can explore the data here.

df = pd.read_csv(data_path)

df

Upload the data to the instance and set it on the project.

In [ ]:
datasource = client.new_csv_datasource(csv=data_path, name=f"patient_data_{uuid.uuid4()}", clear_if_exists=True)

In [ ]:
project.set_datasource(datasource)

### Task Definition

In this notebook, we will run a very simple computation: computing the average value of each column.

In [ ]:
from tuneinsight.computations import Aggregation

In [ ]:
task = Aggregation(
    project=project,
    columns=list(df.columns),
    average=True,
)

### Authorization and summary

Because we are working in a federated setting, each participant needs to authorize the project before it can be run. In this workshop, we have configured the instances to automatically approve aggregation and machine learning projects. In the real world, this means that the project would go through manual review at each participating instance.

In [ ]:
project.request_authorization()

We can now show a summary of the shared project's state.

In [ ]:
project.display_overview()

In [ ]:
project.display_datasources()

## Run the computation

The project is properly configured and each participant has loaded their data and authorized the project. We can now run a federated aggregation.

In [ ]:
results = task.run()

In [ ]:
results